In [7]:
#Import libraries
from pyhive import presto
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
import h3
from datetime import datetime
from functools import reduce
import json
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', 500)

In [8]:
#Define presto credentials
presto_host = 'presto-gateway.serving.data.plectrum.dev'
presto_port = '80'
username = 'aditya.bhattar@rapido.bike'

#Create connection to presto host
connection = presto.connect(presto_host,presto_port,username = username)

In [43]:
bangalore_auto = pd.read_csv('bangalore_auto.csv')
bangalore_auto_hexes = bangalore_auto['pickup_location_hex_8'].unique().tolist()
print(bangalore_auto_hexes)
print(len(bangalore_auto_hexes))

['8861892433fffff', '886189251bfffff', '88618925c3fffff', '8861892589fffff', '886189242dfffff', '8861892421fffff', '88618925d3fffff', '88618925cdfffff', '88618925d1fffff', '886189255bfffff', '8861892553fffff', '88618925d5fffff', '88618925e3fffff', '88618925ebfffff', '88618924adfffff', '8861892513fffff', '88618924e7fffff', '88618925c5fffff', '88618925dbfffff', '88618925ddfffff', '88618925c7fffff', '88618924a5fffff', '88618924e5fffff', '88618925c9fffff', '8861892599fffff', '88618925d9fffff', '8861892437fffff', '88618925e9fffff', '88618925d7fffff', '88618925c1fffff', '8861892591fffff', '88618925cbfffff', '8861892581fffff', '8861892431fffff', '886189259dfffff', '886189258dfffff', '886189258bfffff', '886189259bfffff', '8861892435fffff', '8861892517fffff', '88618925e1fffff', '88618925edfffff', '88618925e7fffff']
43


In [37]:
#Query data from order_logs_snapshot

q = """
select order_id, accept_to_pickup_distance, city_name, customer_location_hex_8, customer_location_latitude, 
customer_location_longitude, hhmmss, hour, quarter_hour, service_obj_service_name, time_bucket, weekday, yyyymmdd,
order_status, cancel_reason, eta, map_riders, pickup_location_hex_8, captain_id
from orders.order_logs_snapshot
where yyyymmdd = '20230417'
and city_name in ('Bangalore')
and service_obj_service_name = 'Auto'
and pickup_location_hex_8 in ('8861892433fffff', '886189251bfffff', '88618925c3fffff', '8861892589fffff', '886189242dfffff', '8861892421fffff', '88618925d3fffff',
 '88618925cdfffff', '88618925d1fffff', '886189255bfffff', '8861892553fffff', '88618925d5fffff', '88618925e3fffff', '88618925ebfffff', '88618924adfffff',
   '8861892513fffff', '88618924e7fffff', '88618925c5fffff', '88618925dbfffff', '88618925ddfffff', '88618925c7fffff', '88618924a5fffff', '88618924e5fffff',
     '88618925c9fffff', '8861892599fffff', '88618925d9fffff', '8861892437fffff', '88618925e9fffff', '88618925d7fffff', '88618925c1fffff', '8861892591fffff',
       '88618925cbfffff', '8861892581fffff', '8861892431fffff', '886189259dfffff', '886189258dfffff', '886189258bfffff', '886189259bfffff', '8861892435fffff',
         '8861892517fffff', '88618925e1fffff', '88618925edfffff', '88618925e7fffff')
and hour in ('17', '18', '19', '20', '21')
"""


#Load data into pandas table
df_copy = pd.read_sql(q, connection)

In [38]:
df = df_copy.copy()
df.head()

,order_id,accept_to_pickup_distance,city_name,customer_location_hex_8,customer_location_latitude,customer_location_longitude,hhmmss,hour,quarter_hour,service_obj_service_name,time_bucket,weekday,yyyymmdd,order_status,cancel_reason,eta,map_riders,pickup_location_hex_8,captain_id
0,643d66fee0382b532b6874f1,0.038,Bangalore,88618925ebfffff,12.941095,77.617676,210422,21,2100,Auto,21:00,1,20230417,customerCancelled,Taking longer than expected,2.0,"[""5c2718134a267149c75ffff9"",""610025756d6070ee8...",88618925ebfffff,610025756d6070ee89476f62
1,643d67207088525d2a238fcd,NaN,Bangalore,88618925c1fffff,12.927151,77.615936,210456,21,2100,Auto,21:00,1,20230417,customerCancelled,order cancelled before rider accepted,3.0,"[""61726b73fbce667de79f0cab"",""6245076d5fd0225ad...",88618925c1fffff,63ecc463e7e4973fd6818e9c
2,643d676b24bf5168f0ffcc61,1.084,Bangalore,886189242dfffff,12.908752,77.647552,210611,21,2100,Auto,21:00,1,20230417,dropped,None,3.0,"[""5e29328bc2db0797a728178d"",""60f10b288bb15d707...",886189242dfffff,60ed6b5e13ae1d1a1ab57ecd
3,643d68f7aab9df2fa9d2b511,NaN,Bangalore,886189255bfffff,12.914542,77.651718,211247,21,2100,Auto,21:00,1,20230417,customerCancelled,order cancelled before rider accepted,2.0,"[""6121e51634b93d287b2e823a"",""60f10b288bb15d707...",886189255bfffff,62e238a8cb6157e6425c71df
4,643d6a89c314db3050e1f28a,NaN,Bangalore,8861892513fffff,12.934195,77.629846,211929,21,2115,Auto,21:00,1,20230417,customerCancelled,order cancelled before rider accepted,2.0,"[""60f27c328f3479594d186c4b"",""61c2ae5917dd6a4ed...",8861892513fffff,60e69d4509aa996512423fe1


In [39]:
df['map_riders'] = df['map_riders'].apply(lambda x: json.loads(x))
df_filter = df[df['captain_id'] != '']
get_index = lambda x: x['map_riders'].index(x['captain_id'])
df_filter['captain_rank'] = df_filter.apply(get_index, axis=1)
df_filter.head()

ValueError: '6030fd274ef13b483a333b5e' is not in list

In [40]:
df_filter['number_of_mapped_captains'] = df['map_riders'].apply(lambda x: len(x))
df_filter.groupby(['hour'])['number_of_mapped_captains'].median()

hour
17    17.0
18    13.0
19    11.0
20    14.0
21    17.0
Name: number_of_mapped_captains, dtype: float64

In [41]:
df_filter.groupby(['hour'])['number_of_mapped_captains'].quantile(0.9)


hour
17    36.0
18    22.0
19    19.0
20    27.0
21    34.0
Name: number_of_mapped_captains, dtype: float64

In [42]:
df.groupby(['hour'])['order_id'].nunique()

hour
17    4317
18    8235
19    9288
20    4696
21    2528
Name: order_id, dtype: int64

In [17]:
df_net = df[df['order_status'] == 'dropped']
net_orders = df_net['order_id'].unique().tolist()

In [18]:
#Query data from order_logs_snapshot

q = """
select order_id, accept_to_pickup_distance, captain_location_hex_8, captain_location_latitude,
captain_location_longitude, city_name, customer_location_hex_8, drop_location_hex_8, drop_location_latitude, drop_location_longitude, customer_location_latitude, 
customer_location_longitude, hhmmss, hour, quarter_hour, service_obj_service_name, time_bucket, weekday, yyyymmdd,
order_status, cancel_reason, updated_epoch, event_type, captain_id, spd_fraud_flag, pickup_location_hex_8
from orders.order_logs_immutable
where yyyymmdd = '20230422'
and service_obj_service_name = 'Auto'
and event_type in ('accepted', 'rider_busy', 'rider_reject', 'dropped')
and pickup_location_hex_8 in ('8861892433fffff', '886189251bfffff', '88618925c3fffff', '8861892589fffff', '886189242dfffff', '8861892421fffff', '88618925d3fffff',
 '88618925cdfffff', '88618925d1fffff', '886189255bfffff', '8861892553fffff', '88618925d5fffff', '88618925e3fffff', '88618925ebfffff', '88618924adfffff',
   '8861892513fffff', '88618924e7fffff', '88618925c5fffff', '88618925dbfffff', '88618925ddfffff', '88618925c7fffff', '88618924a5fffff', '88618924e5fffff',
     '88618925c9fffff', '8861892599fffff', '88618925d9fffff', '8861892437fffff', '88618925e9fffff', '88618925d7fffff', '88618925c1fffff', '8861892591fffff',
       '88618925cbfffff', '8861892581fffff', '8861892431fffff', '886189259dfffff', '886189258dfffff', '886189258bfffff', '886189259bfffff', '8861892435fffff',
         '8861892517fffff', '88618925e1fffff', '88618925edfffff', '88618925e7fffff')
and hour in ('17', '18', '19', '20', '21')
"""

#Load data into pandas table
df_apr_copy = pd.read_sql(q, connection)

In [19]:
df_apr = df_apr_copy.copy()
df_apr.head()

,order_id,accept_to_pickup_distance,captain_location_hex_8,captain_location_latitude,captain_location_longitude,city_name,customer_location_hex_8,drop_location_hex_8,drop_location_latitude,drop_location_longitude,customer_location_latitude,customer_location_longitude,hhmmss,hour,quarter_hour,service_obj_service_name,time_bucket,weekday,yyyymmdd,order_status,cancel_reason,updated_epoch,event_type,captain_id,spd_fraud_flag,pickup_location_hex_8
0,6443cb452a803956b68dd1ec,NaN,88618925d1fffff,12.918883,77.595757,Bangalore,88618925d3fffff,8861892597fffff,12.943953,77.576145,12.919811,77.588543,172549,17,1715,Auto,17:00,6,20230422,new,None,1682164562767,rider_reject,6268ebf0e96179ef89599e9f,None,88618925d3fffff
1,6443cb4c306023341f538935,NaN,8861892513fffff,12.932407,77.628863,Bangalore,8861892513fffff,8861892589fffff,12.931755,77.609665,12.934622,77.635513,172556,17,1715,Auto,17:00,6,20230422,new,None,1682164567413,rider_reject,610781416d6bc05decc5b447,None,8861892513fffff
2,6443cb9c4ec9443f1ebb2d79,NaN,88618925c5fffff,12.933548,77.621600,Bangalore,88618925c5fffff,8861892533fffff,12.956213,77.638824,12.934659,77.623734,172716,17,1715,Auto,17:00,6,20230422,new,None,1682164821347,rider_busy,6229dc5ab46ca7e2e499d1c0,None,88618925c5fffff
3,6443cbb9d12d25505285da77,NaN,886189242dfffff,12.908879,77.644279,Bangalore,886189242dfffff,8861892589fffff,12.930928,77.606176,12.908940,77.644043,172745,17,1715,Auto,17:00,6,20230422,new,None,1682164837139,rider_busy,5f81b251117635a21e11e74c,None,886189242dfffff
4,6443cf9d2882bc25c18c8699,NaN,8861892553fffff,12.921118,77.645127,Bangalore,886189255bfffff,8861892ed3fffff,12.973466,77.620408,12.914026,77.649568,174421,17,1730,Auto,17:59,6,20230422,new,None,1682165701365,rider_reject,62a6c7b2bf19e628a77268a1,None,886189255bfffff


In [20]:
#Filter the apr only on net_orders
df_apr_net = df_apr[df_apr['order_id'].isin(net_orders)]
df_apr_net.head()

,order_id,accept_to_pickup_distance,captain_location_hex_8,captain_location_latitude,captain_location_longitude,city_name,customer_location_hex_8,drop_location_hex_8,drop_location_latitude,drop_location_longitude,customer_location_latitude,customer_location_longitude,hhmmss,hour,quarter_hour,service_obj_service_name,time_bucket,weekday,yyyymmdd,order_status,cancel_reason,updated_epoch,event_type,captain_id,spd_fraud_flag,pickup_location_hex_8
5,6443cc3b93ebb168118a77d7,NaN,8861892591fffff,12.932901,77.580268,Bangalore,8861892591fffff,8860145a69fffff,12.941876,77.567920,12.936585,77.579575,172955,17,1715,Auto,17:00,6,20230422,new,None,1682164800482,rider_reject,62499b5fb4ab1b807e439820,None,8861892591fffff
8,6443cff9306023341f539379,NaN,88618924e5fffff,12.905213,77.593465,Bangalore,8861892433fffff,8861892735fffff,12.911452,77.676315,12.906995,77.606247,174553,17,1745,Auto,17:59,6,20230422,new,None,1682165761304,rider_reject,60e6bb0e09aa997b87425526,None,8861892433fffff
29,6443d287e8dca00213656cd5,NaN,8861892427fffff,12.916960,77.630028,Bangalore,886189251bfffff,8861892557fffff,12.923633,77.649544,12.921909,77.635788,175647,17,1745,Auto,17:59,6,20230422,new,None,1682166417101,rider_reject,5e46a7dbab0265115e27bfb7,None,886189251bfffff
30,6443d2992a803956b68de146,NaN,8861892599fffff,12.928634,77.582077,Bangalore,8861892599fffff,88618924ebfffff,12.885006,77.587544,12.924762,77.581589,175705,17,1745,Auto,17:59,6,20230422,new,None,1682166525190,rider_reject,628ba2fbcf1a86d01b125a40,None,8861892599fffff
31,6443d2a244636e16f081619f,1.477,8860145a4dfffff,12.929787,77.570602,Bangalore,8861892437fffff,8860145a4dfffff,12.929787,77.570602,12.908487,77.611549,175714,17,1745,Auto,17:59,6,20230422,dropped,None,1682168184779,dropped,612a68e7890209a7ad5b3417,False,8861892437fffff


In [21]:
#Create a count column
df_apr_net['count'] = 1

#Create a column to identify total pings and accepted ping
df_apr_net['total_pings'] = np.where(df_apr_net['event_type'].isin(['accepted', 'rider_busy', 'rider_reject']), 1, 0)
df_apr_net['accepted_pings'] = np.where(df_apr_net['event_type'].isin(['accepted']), 1, 0)
df_me = df_apr_net[(df_apr_net['order_status'] == 'dropped') & (df_apr_net['spd_fraud_flag'] != True)]

#Groupby on date and hour to get the APR and ME on an hourly basis
apr_hyderabad = df_apr_net.groupby(['yyyymmdd', 'hour'])[['total_pings', 'accepted_pings']].sum().reset_index()
apr_hyderabad['APR'] = round(apr_hyderabad['accepted_pings']/apr_hyderabad['total_pings'], 4)
me_hyderabad = df_me.groupby(['yyyymmdd', 'hour'])['order_id'].nunique().reset_index()
me_hyderabad = me_hyderabad.rename(columns={'order_id':'net_orders'})
apr_hyderabad = apr_hyderabad.merge(me_hyderabad, on = ['yyyymmdd', 'hour'], how = 'left')
apr_hyderabad['ME'] = round(apr_hyderabad['total_pings']/apr_hyderabad['net_orders'], 2)

#Merge APR and ME with metrics_df
apr_hyderabad

,yyyymmdd,hour,total_pings,accepted_pings,APR,net_orders,ME
0,20230422,17,6716,1254,0.1867,1195,5.62
1,20230422,18,4387,1060,0.2416,1026,4.28
2,20230422,19,2542,870,0.3423,851,2.99
3,20230422,20,2501,788,0.3151,766,3.27
4,20230422,21,3115,674,0.2164,642,4.85


In [32]:
#Create a count column
df_apr['count'] = 1

#Create a column to identify total pings and accepted ping
df_apr['total_pings'] = np.where(df_apr['event_type'].isin(['accepted', 'rider_busy', 'rider_reject']), 1, 0)
df_apr['accepted_pings'] = np.where(df_apr['event_type'].isin(['accepted']), 1, 0)
df_me = df_apr[(df_apr['order_status'] == 'dropped') & (df_apr['spd_fraud_flag'] != True)]

#Groupby on date and hour to get the APR and ME on an hourly basis
apr_hyderabad = df_apr.groupby(['yyyymmdd', 'hour'])[['total_pings', 'accepted_pings']].sum().reset_index()
apr_hyderabad['APR'] = round(apr_hyderabad['accepted_pings']/apr_hyderabad['total_pings'], 4)
me_hyderabad = df_me.groupby(['yyyymmdd', 'hour'])['order_id'].nunique().reset_index()
me_hyderabad = me_hyderabad.rename(columns={'order_id':'net_orders'})
apr_hyderabad = apr_hyderabad.merge(me_hyderabad, on = ['yyyymmdd', 'hour'], how = 'left')
apr_hyderabad['ME'] = round(apr_hyderabad['total_pings']/apr_hyderabad['net_orders'], 2)

#Merge APR and ME with metrics_df
apr_hyderabad

,yyyymmdd,hour,total_pings,accepted_pings,APR,net_orders,ME
0,20230422,17,57731,1743,0.0302,1195,48.31
1,20230422,18,54446,1578,0.0290,1026,53.07
2,20230422,19,50669,1327,0.0262,851,59.54
3,20230422,20,53184,1256,0.0236,766,69.43
4,20230422,21,53648,1111,0.0207,642,83.56
